# Agent Sandbox 自定义沙箱工具教程

## 概述

本教程将指导您如何使用腾讯云 Python SDK 创建和管理**自定义沙箱工具**。自定义沙箱工具允许您使用自己的 OCI 镜像来创建特定用途的沙箱环境。

### 什么是自定义沙箱工具？

- **类型**: `custom` - 使用自定义 OCI 镜像的沙箱工具
- **用途**: 为特定业务场景提供定制化的执行环境
- **优势**: 完全控制运行环境，支持任意软件栈和依赖

### 支持的沙箱工具类型

- `browser` - 浏览器沙箱
- `code-interpreter` - 代码解释器沙箱
- `custom` - 自定义沙箱（本教程重点）

### 主要功能

1. 创建自定义沙箱工具（基于 OCI 镜像）
2. 管理沙箱工具生命周期
3. 启动和管理沙箱实例
4. 监控和维护工具状态

## 1. 环境准备

### 1.1 安装和验证 SDK

In [ ]:
%pip install 'tencentcloud-sdk-python>=3.1.32'
# 验证 SDK 安装
try:
    import tencentcloud
    from tencentcloud.ags.v20250920 import ags_client, models
    print(f"✅ 腾讯云 SDK 版本: {tencentcloud.__version__}")
    print("✅ AGS 模块导入成功")
    print("✅ SDK 版本: v20250920")
except ImportError as e:
    print(f"❌ 导入失败: {e}")
    print("请确保已安装腾讯云 SDK: pip install 'tencentcloud-sdk-python>=3.1.32'")

### 1.2 配置访问凭证

在使用 Agent Sandbox API 之前，您需要获取访问凭证：

1. 登录 [腾讯云控制台](https://console.cloud.tencent.com/cam/capi)
2. 进入「访问管理」→「API 密钥管理」
3. 创建或查看您的 SecretId 和 SecretKey

In [ ]:
import os
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.ags.v20250920 import ags_client, models

# 配置访问凭证（推荐使用环境变量）
if not os.getenv("TENCENTCLOUD_SECRET_ID"):
    os.environ["TENCENTCLOUD_SECRET_ID"] = "your_secret_id"  # 请替换为您的 SecretId
if not os.getenv("TENCENTCLOUD_SECRET_KEY"):
    os.environ["TENCENTCLOUD_SECRET_KEY"] = "your_secret_key"  # 请替换为您的 SecretKey
if not os.getenv("TENCENTCLOUD_REGION"):
    os.environ["TENCENTCLOUD_REGION"] = "ap-guangzhou"

# 创建认证对象
cred = credential.Credential(
    os.environ.get("TENCENTCLOUD_SECRET_ID"),
    os.environ.get("TENCENTCLOUD_SECRET_KEY")
)

# 配置 HTTP 选项
http_profile = HttpProfile()
http_profile.endpoint = "ags.tencentcloudapi.com"  # Agent Sandbox 服务域名

# 配置客户端选项
client_profile = ClientProfile()
client_profile.httpProfile = http_profile

# 创建 AGS 客户端
client = ags_client.AgsClient(cred, os.getenv("TENCENTCLOUD_REGION"), client_profile)

print("✅ 访问凭证配置完成")
print(f"📍 服务端点: {http_profile.endpoint}")
print(f"🌍 地域: {os.getenv('TENCENTCLOUD_REGION')}")
print("✅ AGS 客户端创建成功")

### 1.3 配置沙箱工具参数

为了保持配置的一致性和可维护性，我们将所有沙箱工具相关的配置参数统一放在环境变量中：

In [ ]:
# ========================================
# 沙箱工具配置参数（统一管理）
# ========================================

# 基础配置
os.environ["SANDBOX_TOOL_NAME"] = "custom-dev-aaa"
os.environ["SANDBOX_TOOL_TYPE"] = "custom"
os.environ["SANDBOX_DESCRIPTION"] = ""
os.environ["SANDBOX_DEFAULT_TIMEOUT"] = "30m"
# 必须配置，需要该角色拥有 TCR 以下接口的权限 DescribeInstances、DescribeImages、CreateInstanceToken
os.environ["SANDBOX_ROLE_ARN"] = "qcs::cam::uin/3321337994:roleName/tcr-full-ags"

# 镜像配置
os.environ["SANDBOX_IMAGE"] = "tkeai.tencentcloudcr.com/finofliu/nginx:latest"  # 请替换为您的实际镜像
os.environ["SANDBOX_IMAGE_REGISTRY_TYPE"] = "enterprise"  # enterprise 或 personal
os.environ["SANDBOX_COMMAND"] = '["nginx"]'
os.environ["SANDBOX_ARGS"] = '["-g", "daemon off;"]'
os.environ["SANDBOX_PORTS"] = '[{"Name": "http", "Port": 80, "Protocol": "TCP"}]'

# 资源配置
os.environ["SANDBOX_CPU"] = "1"  # CPU 核心数
os.environ["SANDBOX_MEMORY"] = "1Gi"  # 内存大小

# 网络配置
os.environ["SANDBOX_NETWORK_MODE"] = "PUBLIC"  # PUBLIC, VPC, SANDBOX

# 环境变量配置（JSON 格式）
os.environ["SANDBOX_ENV_VARS"] = '[{"name": "LANG", "value": "en_US.UTF-8"}, {"name": "PATH", "value": "/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"}, {"name": "DEBIAN_FRONTEND", "value": "noninteractive"}]'

# 标签配置（JSON 格式）
os.environ["SANDBOX_TAGS"] = '[]'

# 探针配置（健康检查 - 必须配置）
os.environ["SANDBOX_PROBE_PATH"] = "/"  # 健康检查路径
os.environ["SANDBOX_PROBE_PORT"] = "80"  # 健康检查端口
os.environ["SANDBOX_PROBE_SCHEME"] = "HTTP"  # 协议 HTTP/HTTPS
os.environ["SANDBOX_PROBE_READY_TIMEOUT_MS"] = "30000"  # 就绪超时（毫秒）
os.environ["SANDBOX_PROBE_TIMEOUT_MS"] = "1000"  # 单次探测超时（毫秒）
os.environ["SANDBOX_PROBE_PERIOD_MS"] = "100"  # 探测间隔（毫秒）
os.environ["SANDBOX_PROBE_SUCCESS_THRESHOLD"] = "1"  # 成功阈值
os.environ["SANDBOX_PROBE_FAILURE_THRESHOLD"] = "100"  # 失败阈值

# 实例配置
os.environ["SANDBOX_INSTANCE_TIMEOUT"] = "15m"  # 实例超时时间

print("✅ 沙箱工具配置参数设置完成")
print(f"🔧 工具名称: {os.getenv('SANDBOX_TOOL_NAME')}")
print(f"🐳 镜像地址: {os.getenv('SANDBOX_IMAGE')}")
print(f"💾 资源配置: {os.getenv('SANDBOX_CPU')} CPU, {os.getenv('SANDBOX_MEMORY')} 内存")
print(f"🌐 网络模式: {os.getenv('SANDBOX_NETWORK_MODE')}")
print(f"🔍 探针配置: 启用 ({os.getenv('SANDBOX_PROBE_PATH')}:{os.getenv('SANDBOX_PROBE_PORT')})")
print(f"⏱️  默认超时: {os.getenv('SANDBOX_DEFAULT_TIMEOUT')}")

## 2. 自定义沙箱工具管理

### 2.1 创建自定义沙箱工具

根据 SDK 文档，创建自定义沙箱工具需要以下参数：
- `ToolName`: 工具名称（1-50字符，英文、数字、下划线、连接线）
- `ToolType`: 工具类型（custom）
- `CustomConfiguration`: 自定义配置（镜像、环境变量、探针等）
- `NetworkConfiguration`: 网络配置
- `Description`: 描述（最大200字符）
- `DefaultTimeout`: 默认超时时间（格式：5m、300s、1h等）

In [ ]:
import json

def create_custom_sandbox_tool():
    """
    创建自定义沙箱工具（使用环境变量配置）
    
    Returns:
        dict: 创建结果
    """
    try:
        # 创建请求对象
        req = models.CreateSandboxToolRequest()
        
        # 设置基本参数（从环境变量读取）
        req.ToolName = os.getenv("SANDBOX_TOOL_NAME")
        req.ToolType = os.getenv("SANDBOX_TOOL_TYPE")
        req.Description = os.getenv("SANDBOX_DESCRIPTION")
        req.DefaultTimeout = os.getenv("SANDBOX_DEFAULT_TIMEOUT")
        req.RoleArn = os.getenv("SANDBOX_ROLE_ARN")
        
        # 创建自定义配置对象
        custom_config = models.CustomConfiguration()
        custom_config.Image = os.getenv("SANDBOX_IMAGE")
        custom_config.ImageRegistryType = os.getenv("SANDBOX_IMAGE_REGISTRY_TYPE")
        custom_config.Command = json.loads(os.getenv("SANDBOX_COMMAND", '[]'))
        custom_config.Args= json.loads(os.getenv("SANDBOX_ARGS", '[]'))
        custom_config.Ports = json.loads(os.getenv("SANDBOX_PORTS", '[]'))
        
        # 设置环境变量（从配置中解析）
        env_vars_config = json.loads(os.getenv("SANDBOX_ENV_VARS", '[]'))
        env_vars = []
        for env_config in env_vars_config:
            env_var = models.EnvVar()
            env_var.Name = env_config["name"]
            env_var.Value = env_config["value"]
            env_vars.append(env_var)
        custom_config.Env = env_vars
        
        # 设置资源配置（从环境变量读取）
        resource_config = models.ResourceConfiguration()
        resource_config.CPU = os.getenv("SANDBOX_CPU")
        resource_config.Memory = os.getenv("SANDBOX_MEMORY")
        custom_config.Resources = resource_config
        
        # 设置探针配置（健康检查 - 必须配置）
        probe_config = models.ProbeConfiguration()
        
        # HTTP GET 探测配置
        http_get = models.HttpGetAction()
        http_get.Path = os.getenv("SANDBOX_PROBE_PATH")
        http_get.Port = int(os.getenv("SANDBOX_PROBE_PORT"))
        http_get.Scheme = os.getenv("SANDBOX_PROBE_SCHEME")
        probe_config.HttpGet = http_get
        
        # 探针时间配置
        probe_config.ReadyTimeoutMs = int(os.getenv("SANDBOX_PROBE_READY_TIMEOUT_MS"))
        probe_config.ProbeTimeoutMs = int(os.getenv("SANDBOX_PROBE_TIMEOUT_MS"))
        probe_config.ProbePeriodMs = int(os.getenv("SANDBOX_PROBE_PERIOD_MS"))
        probe_config.SuccessThreshold = int(os.getenv("SANDBOX_PROBE_SUCCESS_THRESHOLD"))
        probe_config.FailureThreshold = int(os.getenv("SANDBOX_PROBE_FAILURE_THRESHOLD"))
        
        custom_config.Probe = probe_config
        
        req.CustomConfiguration = custom_config
        
        # 创建网络配置对象（从环境变量读取）
        network_config = models.NetworkConfiguration()
        network_config.NetworkMode = os.getenv("SANDBOX_NETWORK_MODE")
        req.NetworkConfiguration = network_config
        
        # 设置标签（从配置中解析）
        tags_config = json.loads(os.getenv("SANDBOX_TAGS", '[]'))
        tags = []
        for tag_config in tags_config:
            tag = models.Tag()
            tag.Key = tag_config["key"]
            tag.Value = tag_config["value"]
            tags.append(tag)
        req.Tags = tags
        
        # 发送创建请求
        resp = client.CreateSandboxTool(req)
        
        print(f"✅ 自定义沙箱工具创建成功")
        print(f"🔧 工具名称: {req.ToolName}")
        print(f"🐳 镜像地址: {custom_config.Image}")
        print(f"📝 描述: {req.Description}")
        print(f"⏱️  默认超时: {req.DefaultTimeout}")
        print(f"💾 资源配置: {resource_config.CPU} CPU, {resource_config.Memory} 内存")
        print(f"🌐 网络模式: {network_config.NetworkMode}")
        if hasattr(custom_config, 'Probe') and custom_config.Probe:
            print(f"🔍 探针配置: ({custom_config.Probe.HttpGet.Path}:{custom_config.Probe.HttpGet.Port})")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 创建失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None

# 创建自定义沙箱工具
print("=== 创建自定义沙箱工具 ===")
custom_tool_result = create_custom_sandbox_tool()
tool_id = custom_tool_result.ToolId

### 2.2 查询沙箱工具详情

In [ ]:
def list_sandbox_tools(tool_id=None, limit=1, offset=0):
    """
    查询沙箱工具列表
    
    Args:
        limit (int): 返回数量限制（最大100）
        offset (int): 偏移量
    
    Returns:
        response: 查询结果
    """
    try:
        # 创建请求对象
        req = models.DescribeSandboxToolListRequest()
        req.ToolIds = [tool_id]
        
        # 设置分页参数
        req.Limit = limit
        req.Offset = offset
        
        resp = client.DescribeSandboxToolList(req)
        
        print(f"📋 沙箱工具列表查询成功")
        print(f"📊 限制数量: {limit}，偏移量: {offset}")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 查询失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None


print("\n=== 查询自定义类型沙箱工具 ===")
custom_tools = list_sandbox_tools(tool_id=tool_id)

### 2.5 删除沙箱工具

In [ ]:
def delete_sandbox_tool(tool_id=None):
    """
    删除沙箱工具
    
    Args:
        tool_id (str): 工具ID
    
    Returns:
        response: 删除结果
    """
    try:
        # 创建请求对象
        req = models.DeleteSandboxToolRequest()
        
        # 设置工具标识
        if tool_id:
            req.ToolId = tool_id
        
        # 发送删除请求
        resp = client.DeleteSandboxTool(req)
        
        print(f"✅ 沙箱工具删除成功")
        if tool_id:
            print(f"🆔 已删除工具ID: {tool_id}")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 删除失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None

# 示例：删除工具（谨慎操作）
# delete_result = delete_sandbox_tool(tool_id)
print("ℹ️  删除功能示例（取消注释上方代码执行，谨慎操作）")

## 3. 沙箱实例管理

### 3.1 启动沙箱实例

使用创建的自定义工具启动沙箱实例。根据 SDK 文档，启动实例需要：
- `ToolId` 或 `ToolName`：指定要使用的沙箱工具
- `Timeout`：实例超时时间
- `CustomConfiguration`：实例级别的自定义配置（可选）

In [ ]:
def start_sandbox_instance(tool_name=None, tool_id=None, timeout=None):
    """
    启动沙箱实例（使用环境变量配置）
    
    Args:
        tool_name (str): 工具名称（与 tool_id 二选一，默认使用环境变量）
        tool_id (str): 工具ID（与 tool_name 二选一）
        timeout (str): 超时时间（默认使用环境变量）
    
    Returns:
        response: 启动结果
    """
    try:
        # 使用环境变量作为默认值
        if not tool_name and not tool_id:
            tool_name = os.getenv("SANDBOX_TOOL_NAME")
        if not timeout:
            timeout = os.getenv("SANDBOX_INSTANCE_TIMEOUT")
        
        if not tool_name and not tool_id:
            raise ValueError("tool_name 和 tool_id 至少需要提供一个")
        
        # 创建请求对象
        req = models.StartSandboxInstanceRequest()
        
        # 设置工具标识
        if tool_id:
            req.ToolId = tool_id
        if tool_name:
            req.ToolName = tool_name
        
        # 设置超时时间
        req.Timeout = timeout
        
        # 可选：设置实例级别的自定义配置（从环境变量读取）
        custom_config = models.CustomConfiguration()
        
        # 可以在这里覆盖工具级别的配置
        # 例如：为特定实例设置不同的环境变量或资源配置
        env_vars_config = json.loads(os.getenv("SANDBOX_ENV_VARS", '[]'))
        if env_vars_config:
            env_vars = []
            for env_config in env_vars_config:
                env_var = models.EnvVar()
                env_var.Name = env_config["name"]
                env_var.Value = env_config["value"]
                env_vars.append(env_var)
            custom_config.Env = env_vars
            req.CustomConfiguration = custom_config
        
        # 发送启动请求
        resp = client.StartSandboxInstance(req)
        
        print(f"✅ 沙箱实例启动成功")
        if tool_name:
            print(f"🔧 使用工具: {tool_name}")
        if tool_id:
            print(f"🆔 工具ID: {tool_id}")
        print(f"⏱️  超时时间: {timeout}")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 启动失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None

# 使用环境变量配置启动实例
print("=== 启动沙箱实例 ===")
instance_result = start_sandbox_instance()

### 3.2 查询沙箱实例列表

In [ ]:
def list_sandbox_instances(tool_id=None, tool_name=None, limit=20, offset=0, status=None):
    """
    查询沙箱实例列表
    
    Args:
        tool_id (str): 工具ID过滤（可选）
        tool_name (str): 工具名称过滤（可选）
        limit (int): 返回数量限制（最大100）
        offset (int): 偏移量
    
    Returns:
        response: 查询结果
    """
    try:
        # 创建请求对象
        req = models.DescribeSandboxInstanceListRequest()
        
        # 设置分页参数
        req.Limit = limit
        req.Offset = offset
        
        # 设置工具过滤（如果指定）
        if tool_id:
            req.ToolId = tool_id
        if status:
            req.Filters = [
                models.Filter()
            ]
            req.Filters[0].Name = "Status"
            req.Filters[0].Values = [status]
        
        # 发送查询请求
        resp = client.DescribeSandboxInstanceList(req)
        
        print(f"📋 沙箱实例列表查询成功")
        if tool_id:
            print(f"🔍 过滤工具ID: {tool_id}")
        print(f"📊 限制数量: {limit}，偏移量: {offset}")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 查询失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None

# 查询所有沙箱实例
print("=== 查询所有沙箱实例 ===")
all_instances = list_sandbox_instances()

# 查询特定工具的实例
print("\n=== 查询特定工具的实例 ===")
tool_instances = list_sandbox_instances(tool_id=tool_id)

instance_id = tool_instances.InstanceSet[0].InstanceId

### 3.4 停止沙箱实例

In [ ]:
def stop_sandbox_instance(instance_id):
    """
    停止沙箱实例
    
    Args:
        instance_id (str): 实例ID
    
    Returns:
        response: 停止结果
    """
    try:
        if not instance_id:
            raise ValueError("instance_id 是必需的")
        
        # 创建请求对象
        req = models.StopSandboxInstanceRequest()
        req.InstanceId = instance_id
        
        # 发送停止请求
        resp = client.StopSandboxInstance(req)
        
        print(f"✅ 沙箱实例停止成功")
        print(f"🆔 已停止实例: {instance_id}")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 停止失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None

# 示例：停止实例（需要实际的实例ID）
# stop_result = stop_sandbox_instance("your_instance_id_here")
print("ℹ️  停止实例示例（需要实际实例ID）")

### 3.5 获取沙箱实例访问令牌

In [ ]:
def acquire_sandbox_instance_token(instance_id):
    """
    获取沙箱实例访问令牌
    
    Args:
        instance_id (str): 实例ID
    
    Returns:
        response: 令牌信息
    """
    try:
        if not instance_id:
            raise ValueError("instance_id 是必需的")
        
        # 创建请求对象
        req = models.AcquireSandboxInstanceTokenRequest()
        req.InstanceId = instance_id
        
        # 发送请求
        resp = client.AcquireSandboxInstanceToken(req)
        
        print(f"✅ 沙箱实例令牌获取成功")
        print(f"🆔 实例ID: {instance_id}")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 获取令牌失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None

# 示例：获取实例令牌（需要实际的实例ID）
token_result = acquire_sandbox_instance_token(instance_id=instance_id)
print("ℹ️  获取令牌示例（需要实际实例ID）")

sandbox_access_token = token_result.Token

### 3.6 实际访问沙箱

In [ ]:
import urllib.request
import urllib.parse
import urllib.error
import json

tool_instances = list_sandbox_instances(tool_id=tool_id, status="RUNNING")
instance_id = tool_instances.InstanceSet[0].InstanceId
token_result = acquire_sandbox_instance_token(instance_id=instance_id)
sandbox_access_token = token_result.Token

port = 80 # 修改为实际监听端口
url = f"https://{port}-{instance_id}.{os.getenv("TENCENTCLOUD_REGION")}.tencentags.com"
headers = {
    "X-Access-Token": sandbox_access_token
}


try:
    req = urllib.request.Request(url=url, headers=headers)
    # 发送 GET 请求，获取响应
    with urllib.request.urlopen(req) as response:
        # 处理响应结果
        status_code = response.getcode()  # 获取状态码
        response_headers = response.getheaders()  # 获取响应头
        response_content = response.read()  # 获取二进制响应内容
        response_text = response_content.decode("utf-8")  # 解码为文本格式
        
        print("=== urllib GET 请求响应结果 ===")
        print("状态码：", status_code)
        print("响应头：", response_headers)
        print("响应文本：", response_text)
except urllib.error.HTTPError as e:
    print("HTTP 错误（状态码 4xx/5xx）：", e.code, e.reason)
except urllib.error.URLError as e:
    print("请求失败（网络问题、域名错误等）：", e.reason)
except Exception as e:
    print("未知错误：", e)
